In [19]:
"""
The goal of this threat hunting playbook is to malicious Windows services
"""
from prettytable import PrettyTable
from helpers import vault
from helpers import kolide 
from helpers import vti
import pandas as pd
import getpass 
import os

# Obtain the necessary values
VAULT_ADDR = os.getenv('VAULT_ADDR') or getpass.getpass("Enter Vault URL: ")
VAULT_SECRET_PATH = "/secrets/data/incident-response/jupyter-notebooks"
VAULT_TOKEN = open(os.path.expanduser('~/.vault-token')).read() or os.getenv('VAULT_TOKEN') or getpass.getpass("Enter Vault token: ")

# Osquery query settings
#SELECT 
#    s.name, 
#    CASE WHEN instr(s.path, '"') > 0 THEN substr(s.path, 2, instr(substr(s.path, 2), '"')-1) ELSE split(s.path, ' ', 0) END as imagepath, 
#    a.result as signature, 
#    h.sha256 
#FROM services s 
#    JOIN authenticode a ON imagepath=a.path 
#    JOIN hash h ON imagepath=h.path 
#    WHERE result != 'trusted' 
#    GROUP by imagepath;
#+------+------------------------------------------------+-----------+------------------------------------------------------------------+
#| name | imagepath                                      | signature | sha256                                                           |
#+------+------------------------------------------------+-----------+------------------------------------------------------------------+
#| BtiM | C:\Users\SUPERA~1\AppData\Local\Temp\zthCB.exe | missing   | c4612fc4073a4cfe5179ec6cc0a63eb9e201f0b3b5fb1239958e88a716b83955 |
#+------+------------------------------------------------+-----------+------------------------------------------------------------------+
osquery_labels_to_scan = ["MS Windows"]
osquery_hosts_to_scan = []
osquery_query = """SELECT s.name, CASE WHEN instr(s.path, '"') > 0 THEN substr(s.path, 2, instr(substr(s.path, 2), '"')-1) ELSE split(s.path, ' ', 0) END as imagepath, a.result as signature, h.sha256 FROM services s JOIN authenticode a ON imagepath=a.path JOIN hash h ON imagepath=h.path WHERE result != 'trusted' GROUP by imagepath;"""

# Get and extract Vault secrets
vault_secrets = vault.GetVaultSecrets(VAULT_ADDR, VAULT_SECRET_PATH, VAULT_TOKEN)
VTI_API_KEY = vault_secrets['vti-api-key']

# Obtain/generate Kolide URL
KOLIDE_URL = str()
if os.getenv('KOLIDE_URL') is not None:
    KOLIDE_URL = input("Enter Kolide URL: ")
    
# Obtain/generate Kolide JWT
kolide_jwt = str()
if os.getenv('KOLIDE_TOKEN') is not None:
    username = input("Enter Kolide username: ")
    password = getpass.getpass("Enter Kolide password: ")
    kolide_jwt = kolide.AuthenticateToKolide(KOLIDE_URL, username, password)
    
# Submit query
kolide_query_campaign_id = kolide.CreateLiveQuery(KOLIDE_URL, kolide_jwt, osquery_query, hosts=osquery_hosts_to_scan, labels=osquery_labels_to_scan)

# Retrieve results
print ("Getting results from Kolide")
results = kolide.GetKolideLiveQueryResults(KOLIDE_URL, kolide_jwt, kolide_query_campaign_id)
print ("Received results from Kolide")


# Generate results table
x = PrettyTable()
x.field_names = ['Hostname','Service Name','Signature','Malicious Score', 'SHA256 file hash', 'File path']
x._max_width = {"SHA256 file hash" : 20, "Field 3" : 25}
for result in results:
    for row in result['data']['rows']:
        # Get VTI result for SHA256 fiel hash
        vti_result = vti.GetVTIresults(VTI_API_KEY, row['sha256'])

        if vti_result.get('error', None):
            x.add_row([row['host_hostname'], row['name'], row['signature'], None, row['sha256'],row['imagepath']])
        else:
            malicious_score = vti_result['data']['attributes']['last_analysis_stats']['malicious']
            if malicious_score > 20:
                x.add_row([row['host_hostname'], row['name'], row['signature'], malicious_score, row['sha256'],row['imagepath']])
print(x)

        

https://vault.hackinglab.local/v1/secrets/data/incident-response/jupyter-notebooks
Enter Kolide URL: https://kolide.hackinglab.local:8443
Enter Kolide username: admin
Enter Kolide password: ········


/Users/thunderwagon/Development/Vault-Jupyter-Notebooks/playbooks/venv/lib/python3.7/site-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'kolide.hackinglab.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/Users/thunderwagon/Development/Vault-Jupyter-Notebooks/playbooks/venv/lib/python3.7/site-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'kolide.hackinglab.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


Getting results from Kolide
Sending JWT
Sending campaign ID
Receiving...
Host DESKTOP-569UBBQ:DC876CDA-DAAE-4823-97A2-E721EFC33227 has submitted results
Received the following number of results: 1/1
Close socket
Received results from Kolide
+-----------------+--------------+-----------+-----------------+----------------------+------------------------------------------------+
|     Hostname    | Service Name | Signature | Malicious Score |   SHA256 file hash   |                   File path                    |
+-----------------+--------------+-----------+-----------------+----------------------+------------------------------------------------+
| DESKTOP-569UBBQ |   YXIeYOUo   |  missing  |       None      | 3c0e49b65ba7748a6244 | C:\Users\SUPERA~1\AppData\Local\Temp\TZoMb.exe |
|                 |              |           |                 | b01916f10157c84e204f |                                                |
|                 |              |           |                 | 3a8880578

In [16]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
